In [2]:
import sys
#sys.path.append("../../tutorials/fastai/old/") # go to parent dir

#from fastai.imports import *
import pandas as pd
import numpy as np
from structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.metrics import f1_score

In [3]:
PATH = "datasets/raw"

In [4]:
!ls {PATH}

pyhs-raw-lead1	pyhs-raw-lead2-HRV  pyhs-raw-leadAVR
pyhs-raw-lead2	pyhs-raw-lead3


In [5]:
lead_1 = pd.read_feather(f'{PATH}/pyhs-raw-lead1')
lead_1

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_RR,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,label
0,64,1,8.203281,4460.777778,1.906904e+05,84.166667,77.666667,-32.750000,-46.666667,4017.0,...,8.233628e+06,3.074513e+10,0.326896,0.452789,-1.114599,-1.285166,4617.666667,5106.166667,300.261749,PAC\n
1,24,1,2.150430,9963.953488,1.612913e+05,183.904762,44.047619,-38.190476,-212.047619,9914.0,...,3.436726e+07,1.481045e+10,0.008229,-0.012282,-1.196234,-1.955239,10309.619048,10776.285714,252.051913,Normal\n
2,59,0,1.054852,7031.666667,2.289459e+05,120.466667,58.733333,-36.200000,-130.266667,6993.0,...,1.704473e+07,4.754209e+10,0.016040,0.102263,-1.178660,-1.859521,6736.266667,7255.600000,247.390310,PAC\n
3,14,1,7.645041,6695.090909,4.049139e+06,56.714286,57.190476,-148.619048,-121.952381,6660.0,...,1.752821e+07,8.191544e+11,0.031014,-4.325836,-1.220443,16.835113,6795.238095,7248.285714,319.844531,Normal\n
4,40,1,2.182612,5615.250000,1.302676e+06,196.000000,40.818182,-218.636364,-259.454545,5640.0,...,1.072786e+07,1.682837e+12,-0.014445,0.045967,-1.216803,-1.940233,5709.272727,6233.818182,237.254954,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,2.200880,5355.391304,6.515502e+05,4.000000,4.600000,-13.800000,-558.100000,5212.0,...,8.467770e+06,5.689022e+11,-0.042738,0.273157,-1.121451,-1.911613,5066.800000,5409.800000,231.441628,AF
6873,78,1,4.201681,5052.095238,2.516323e+06,33.583333,-16.833333,-116.583333,-231.250000,4936.0,...,7.846417e+06,5.001414e+12,-0.059236,-0.246734,-0.964861,-1.885106,4722.000000,5218.000000,254.732016,AF\n
6874,12,1,1.583861,6097.157895,3.228570e+05,276.333333,30.777778,-94.222222,-266.166667,6134.0,...,1.271993e+07,5.643145e+10,-0.021902,0.088967,-1.201353,-1.860108,6067.111111,6543.000000,160.942494,Normal\n
6875,59,1,6.102441,5006.153846,7.516309e+05,-15.900000,13.333333,-36.500000,-34.700000,5632.0,...,9.243060e+06,3.005175e+11,-0.216776,-0.494394,-1.168932,-1.662923,NaN,5240.400000,413.754658,I-AVB\n


In [6]:
lead_2 = pd.read_feather('datasets/raw/pyhs-raw-lead2')
lead_2

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,RSSSD,label
0,64,1,4.601841,5164.823529,3.214186e+06,115.636364,13.181818,-114.363636,-54.090909,5166.0,...,6.085160e+12,0.002240,-0.497951,-1.256533,-1.593309,5029.090909,5508.909091,649.572167,547.989237,PAC\n
1,24,1,1.050210,9965.069767,1.423544e+06,372.800000,-19.380952,-120.000000,-272.333333,9920.0,...,1.638203e+12,0.008188,-0.033648,-1.196573,-1.982349,10304.761905,NaN,501.379620,416.071591,Normal\n
2,59,0,1.054852,7262.193548,7.820505e+05,172.266667,-8.733333,-95.933333,-39.066667,7150.0,...,6.280158e+11,0.008420,0.119925,-1.192760,-1.895794,6725.600000,7256.933333,488.896444,383.210078,PAC\n
3,14,1,4.572735,6977.000000,5.447730e+05,108.761905,44.047619,-72.476190,-157.095238,7275.0,...,4.369731e+10,-0.099483,-2.120175,-1.213845,2.765965,6830.095238,7244.285714,595.538631,343.618955,Normal\n
4,40,1,8.548563,4981.000000,1.035438e+06,25.818182,95.272727,-62.363636,-360.363636,4715.0,...,7.479752e+11,0.134001,-0.406748,-1.146664,-1.832525,5675.818182,6216.545455,572.774044,492.980114,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,2.200880,4668.666667,1.825312e+05,31.083333,44.000000,-50.833333,-249.750000,4152.0,...,3.185242e+10,0.204868,0.142266,-1.288943,-1.781381,4775.666667,5282.166667,518.038609,475.157415,AF
6873,78,1,1.200480,4809.684211,5.056835e+05,3.166667,10.916667,2.000000,-118.500000,4682.0,...,1.562389e+11,-0.024755,-0.474755,-1.099878,-1.683698,4727.500000,5237.000000,555.166842,463.844745,AF\n
6874,12,1,1.583861,6130.918919,1.151930e+06,251.944444,9.500000,-132.388889,-252.000000,6274.0,...,1.133624e+12,-0.045750,-0.039627,-1.238796,-1.978701,6113.444444,6549.333333,335.560228,243.181074,Normal\n
6875,59,1,2.100840,4740.000000,7.664817e+05,22.800000,35.800000,-39.900000,-63.100000,5149.0,...,2.589422e+11,-0.072559,0.008766,-1.329275,-0.491303,4877.200000,5411.000000,781.003103,639.595362,I-AVB\n


In [7]:
lead_2_hrv = pd.read_feather('datasets/raw/pyhs-raw-lead2-HRV')
lead_2_hrv

,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,age,label
0,111.568614,854.000000,79.453690,117.277923,0.093037,0.130642,858.0,32.6172,0.038015,39.5,50.000000,60.000000,288.0,2.500000,64,PAC\n
1,60.510112,933.400000,43.947816,62.130593,0.047084,0.064828,944.0,50.4084,0.053399,80.0,50.000000,70.000000,134.0,4.000000,24,Normal\n
2,170.585417,910.142857,113.762139,177.391354,0.124994,0.187427,920.0,45.9606,0.049957,76.0,21.428571,42.857143,482.0,2.333333,59,PAC\n
3,33.324560,647.100000,29.880287,34.223893,0.046176,0.051498,647.0,35.5824,0.054996,43.5,15.000000,60.000000,108.0,3.333333,14,Normal\n
4,28.975085,925.000000,21.463147,30.688398,0.023203,0.031324,922.0,19.2738,0.020904,23.0,0.000000,70.000000,76.0,2.000000,40,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,316.148067,755.636364,213.348200,332.222415,0.282342,0.418387,816.0,249.0768,0.305241,270.0,90.909091,90.909091,710.0,2.750000,61,AF
6873,163.689951,812.000000,122.735488,172.511384,0.151152,0.201589,748.0,100.8168,0.134782,210.0,63.636364,72.727273,322.0,2.200000,78,AF\n
6874,16.852300,641.176471,17.903335,17.025471,0.027923,0.026283,640.0,11.8608,0.018533,16.0,0.000000,17.647059,68.0,3.400000,12,Normal\n
6875,226.637376,985.777778,135.389972,241.901840,0.137343,0.229907,968.0,53.3736,0.055138,56.0,55.555556,66.666667,438.0,2.250000,59,I-AVB\n


In [8]:
lead_3 = pd.read_feather(f'{PATH}/pyhs-raw-lead3')
lead_3

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_RR,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,label
0,64,1,4.601841,5338.444444,1.978540e+06,34.833333,-10.833333,-79.083333,-2.750000,5586.0,...,8.214257e+06,3.094245e+12,-0.155660,-0.002301,-0.904563,-1.559539,4628.666667,5093.500000,308.684876,PAC\n
1,24,1,1.050210,10673.897436,8.693754e+05,184.100000,-43.523810,-68.428571,-89.095238,10798.0,...,3.208636e+07,5.593285e+11,-0.133419,-0.247003,-1.083598,-1.923511,10282.476190,NaN,280.764839,Normal\n
2,59,0,3.305204,6929.333333,2.865725e+05,57.000000,-52.066667,-65.466667,1.933333,6549.0,...,1.753054e+07,9.142680e+10,0.093603,0.309619,-1.217337,-1.443568,6807.333333,7272.533333,254.652705,PAC\n
3,14,1,3.072306,7017.826087,1.954422e+06,27.476190,36.809524,-34.142857,-810.714286,6968.0,...,1.902774e+07,3.861644e+11,0.021335,-2.819319,-1.233702,6.209456,6694.095238,7197.142857,315.985471,Normal\n
4,40,1,1.091306,5618.333333,4.330715e+05,-11.166667,135.166667,-306.666667,-172.333333,5645.0,...,1.071791e+07,1.624435e+11,-0.014711,0.023107,-1.217563,-1.970314,5337.000000,5717.833333,239.913843,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,1.000400,5173.454545,1.921500e+05,20.583333,36.916667,-204.166667,-29.083333,4615.0,...,8.868226e+06,3.953116e+10,0.110865,0.279354,-1.221931,-1.686269,4845.666667,5336.333333,247.252714,AF
6873,78,1,8.403361,4884.444444,1.576096e+06,133.000000,89.833333,-189.583333,-613.416667,4856.0,...,8.765316e+06,1.338418e+12,0.017874,-0.589572,-1.179053,-1.474632,4742.666667,5112.666667,293.014655,AF\n
6874,12,1,3.084361,6113.600000,9.883849e+05,-1.111111,-1.611111,-108.111111,-39.944444,6329.0,...,1.276475e+07,6.824973e+10,-0.116599,-2.723835,-1.197383,8.419272,6079.333333,6559.555556,307.726810,Normal\n
6875,59,1,2.100840,4779.333333,2.439601e+05,35.300000,66.900000,-135.200000,-43.000000,4652.0,...,8.073701e+06,7.710085e+10,0.053315,1.157110,-1.125432,1.057842,4918.600000,5416.200000,252.569693,I-AVB\n


In [9]:
lead_aVR = pd.read_feather(f'{PATH}/pyhs-raw-leadAVR')
lead_aVR

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_RR,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,label
0,64,1,4.601841,4827.300000,1.009328e+06,57.000000,71.363636,-208.272727,-557.454545,4471.0,...,8.943587e+06,8.413052e+11,0.207141,-0.188590,-1.174447,-1.948377,4978.181818,5407.272727,277.173516,PAC\n
1,24,1,1.050210,9963.348837,5.942735e+05,9.350000,80.476190,-450.095238,25.150000,9918.0,...,3.436611e+07,2.621663e+11,0.008140,-0.032147,-1.196508,-1.981005,10370.285714,NaN,250.719726,Normal\n
2,59,0,1.054852,7031.266667,4.479738e+05,9.266667,62.866667,-441.400000,-20.200000,6991.0,...,1.703914e+07,1.856598e+11,0.016192,0.025717,-1.178556,-1.958938,6789.600000,7323.066667,246.828155,PAC\n
3,14,1,4.572735,6691.727273,1.808194e+06,50.619048,101.571429,-699.476190,-80.904762,6659.0,...,1.754333e+07,1.625181e+11,0.029258,-4.314318,-1.218941,16.769944,6856.285714,7363.619048,320.279640,Normal\n
4,40,1,2.182612,5618.583333,9.752698e+05,7.000000,126.916667,-636.416667,38.750000,5641.0,...,1.073232e+07,9.720858e+11,-0.014015,0.018801,-1.215897,-1.975424,5322.833333,5777.000000,236.845391,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,2.200880,5133.636364,3.797785e+05,2.700000,39.300000,-265.900000,-7.400000,4981.0,...,7.982975e+06,1.740630e+11,-0.003065,0.189844,-1.039107,-1.952672,5079.600000,5494.800000,243.572616,AF
6873,78,1,4.201681,4920.272727,1.093687e+06,56.000000,30.666667,-808.083333,34.333333,5000.0,...,7.553868e+06,1.043178e+12,-0.081801,-0.144951,-1.057208,-1.929049,4848.000000,5279.833333,237.607520,AF\n
6874,12,1,1.583861,6099.157895,5.920009e+05,33.444444,106.888889,-432.611111,48.388889,6136.0,...,1.272005e+07,2.675392e+11,-0.021757,0.015077,-1.201486,-1.976905,6161.555556,6631.333333,161.162332,Normal\n
6875,59,1,1.400560,4558.000000,6.891245e+05,85.100000,17.700000,-572.800000,58.200000,4660.0,...,1.002335e+07,1.415176e+11,0.066869,-1.057442,-1.327494,-0.632230,4927.600000,5490.400000,416.203376,I-AVB\n


In [10]:
for df_raw in [lead_1, lead_2, lead_2_hrv, lead_3, lead_aVR]:
    df_raw.loc[df_raw.label == 'AF\n', 'label'] = 'AF'
    df_raw.loc[df_raw.label == 'I-AVB\n', 'label'] = 'I-AVB'
    df_raw.loc[df_raw.label == 'LBBB\n', 'label'] = 'LBBB'
    df_raw.loc[df_raw.label == 'Normal\n', 'label'] = 'Normal'
    df_raw.loc[df_raw.label == 'PAC\n', 'label'] = 'PAC'
    df_raw.loc[df_raw.label == 'PVC\n', 'label'] = 'PVC'
    df_raw.loc[df_raw.label == 'RBBB\n', 'label'] = 'RBBB'
    df_raw.loc[df_raw.label == 'STD\n', 'label'] = 'STD'
    df_raw.loc[df_raw.label == 'STE\n', 'label'] = 'STE'

In [11]:
labels = pd.get_dummies(lead_aVR['label'])
labels.describe()

,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE
count,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000
mean,0.177548,0.104988,0.028937,0.133488,0.079104,0.091173,0.243566,0.114294,0.026901
std,0.382160,0.306560,0.167642,0.340127,0.269921,0.287877,0.429265,0.318191,0.161807
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
W_PATH = 'datasets/corrected'
lead_1.to_feather(f'{W_PATH}/pyhs-raw-lead1-corrected')
lead_2.to_feather(f'{W_PATH}/pyhs-raw-lead2-corrected')
lead_2_hrv.to_feather(f'{W_PATH}/pyhs-raw-lead2-HRV-corrected')
lead_3.to_feather(f'{W_PATH}/pyhs-raw-lead3-corrected')
lead_aVR.to_feather(f'{W_PATH}/pyhs-raw-leadAVR-corrected')